# Figure 4: NIRCam Grism + Filter Sensitivities ($1^{st}$ order)

***
### Table of Contents

1. [Information](#Information)
2. [Imports](#Imports)
3. [Data](#Data)
4. [Generate the First Order Grism + Filter Sensitivity Plot](#Generate-the-First-Order-Grism-+-Filter-Sensitivity-Plot)
5. [Issues](#Issues)
6. [About this Notebook](#About-this-Notebook)
***

## Information

#### JDox links: 
* [NIRCam Grisms](https://jwst-docs.stsci.edu/display/JTI/NIRCam+Grisms#NIRCamGrisms-Sensitivity)
    * Figure 4. NIRCam grism + filter sensitivities ($1^{st}$ order)

## Imports

In [ ]:
import os
import pylab
import numpy as np
from astropy.io import ascii, fits
from astropy.table import Table
from scipy.optimize import fmin
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
%matplotlib inline

## Data

#### Data Location: 

The data is stored in a NIRCam JDox Box folder here:
[ST-INS-NIRCAM -> JDox -> nircam_grisms](https://stsci.box.com/s/wu9mo54vi957x50rdirlcg9zkkr3xiaw)

#### Load the data

(The next cell assumes you downloaded the data into your ```Users/$(logname)/``` home directory)

In [ ]:
box_directory = "/Users/"+os.environ['LOGNAME']+"/box_data/"   
box_directory

In [ ]:
grism = "R"
mod = "A"
filters = ["F250M","F277W","F300M","F322W2","F335M","F356W","F360M","F410M","F430M","F444W","F460M","F480M"]

In [ ]:
filenames = []
for fil in filters:
    filenames.append(box_directory+"NIRCam.%s.%s.%s.1st.sensitivity.fits" % (fil,grism,mod))                  

In [ ]:
filenames

## Generate the First Order Grism + Filter Sensitivity Plot

### Define some convenience functions

In [ ]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]


def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]


def find_mid(w,s,w0,thr=0.05):
    fct = interp1d(w,s,bounds_error=None,fill_value='extrapolate')
    def func(x):
        #print "x:",x
        return np.abs(fct(x)-thr)
    
    res = fmin(func,w0)
    
    return res[0]

### Create the plots

In [ ]:
f, ax1 = plt.subplots(1, figsize=(15, 10))

NUM_COLORS = len(filters)

cm = pylab.get_cmap('tab10')

grism = "R"
mod = "A"
for i,fname in zip(range(NUM_COLORS),filenames):
    color = cm(1.*i/NUM_COLORS)
    d = fits.open(fname)
    w = d[1].data["WAVELENGTH"]
    s = d[1].data["SENSITIVITY"]/(1e17)
    ax1.plot(w,s,label=fil,lw=4,color=color)
    
ax1.legend(fontsize=16)
miny,maxy = ax1.get_ylim()
minx,maxx = ax1.get_xlim()

ax1.set_ylim(miny,2.15)
ax1.set_xlim(2.1,maxx)
ax1.tick_params(labelsize=18)
f.text(0.5, 0.04, 'Wavelength ($\mu m$)', ha='center', fontsize=22)
f.text(0.03, 0.5, 'Sensitivity  ('+r'$1 \times 10^{17}\ \frac{e^{-} s^{-1}}{erg s^{-1} cm^{-2} A^{-1}}$'+')', va='center', rotation='vertical', fontsize=22)

### Figure option 2: filter name positions

In [ ]:
f, ax1 = plt.subplots(1, figsize=(15, 10))

thr = 0.05 # 5% of peak boundaries

NUM_COLORS = len(filters)

cm = pylab.get_cmap('tab10')

for i,fil,fname in zip(range(NUM_COLORS),filters,filenames):
    color = cm(1.*i/NUM_COLORS)
    d = fits.open(fname)
    w = d[1].data["WAVELENGTH"]
    s = d[1].data["SENSITIVITY"]/(1e17)

    wmin,wmax = np.min(w),np.max(w)
    vg = w<(wmax+wmin)/2.
    w1 = find_mid(w[vg],s[vg],wmin,thr)
    
    vg = w>(wmax+wmin)/2.
    w2 =  find_mid(w[vg],s[vg],wmax,thr)    
    
    if fil == 'F356W':
        ax1.text((w2+w1)/2 -0.04, s[np.where(w == find_nearest(w, (w2+w1)/2))]+0.25, fil, ha='center',color=color,fontsize=16,weight='bold')
    elif fil == 'F335M':
        ax1.text((w2+w1)/2 -0.03, s[np.where(w == find_nearest(w, (w2+w1)/2))]+0.22, fil, ha='center',color=color,fontsize=16,weight='bold')        
    elif fil == 'F460M':
        ax1.text((w2+w1)/2+0.15, s[np.where(w == find_nearest(w, (w2+w1)/2))]+0.12, fil, ha='center',color=color,fontsize=16,weight='bold')        
    elif fil == 'F480M':
        ax1.text((w2+w1)/2+0.15, s[np.where(w == find_nearest(w, (w2+w1)/2))]+0.1, fil, ha='center',color=color,fontsize=16,weight='bold')
    else:
        ax1.text((w2+w1)/2 -0.04, s[np.where(w == find_nearest(w, (w2+w1)/2))]+0.2, fil, ha='center',color=color,fontsize=16,weight='bold')        
    ax1.plot(w,s,label=fil,lw=4,color=color)
    

miny,maxy = ax1.get_ylim()
minx,maxx = ax1.get_xlim()

ax1.set_ylim(miny,2.15)
ax1.set_xlim(2.1,maxx)
ax1.tick_params(labelsize=18)
f.text(0.5, 0.04, 'Wavelength ($\mu m$)', ha='center', fontsize=22)
f.text(0.03, 0.5, 'Sensitivity  ('+r'$1 \times 10^{17}\ \frac{e^{-} s^{-1}}{erg\ s^{-1} cm^{-2} A^{-1}}$'+')', va='center', rotation='vertical', fontsize=22)

## Issues

* None

## About this Notebook
**Authors:**   
Nor Pirzkal & Alicia Canipe

**Updated On:**   
April 10, 2019